In [1]:
import feedparser
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
import fiona
import folium
import os
%matplotlib notebook

CHECKS FOR ALERTS ON MULTIPLE RSS FEEDS

In [2]:
### GDACS GeoRSS
GDACS = 'http://www.gdacs.org/xml/rss.xml'

RSS_URL = [GDACS]

feeds = []
iso3 = []
country = []
etype = []
epid = []
evid = []
guid = []
pubdate = []
bbox = []
coords = []
alevel = []
ascore = []

wfpiso3 = ['CHL', 'IDN', 'PHL', 'SDN', 'SSD', 'DRC', 'BGD', 'CAF', 'BWA', 'TCD', 'KHM', 'IND']

def get_data():
    for url in RSS_URL:
        feeds.append(feedparser.parse(url))

    for feed in feeds:
        for post in feed.entries:
            iso3.append(post.gdacs_iso3)
            country.append(post.gdacs_country)
            etype.append(post.gdacs_eventtype)
            epid.append(post.gdacs_episodeid)
            evid.append(post.gdacs_eventid)
            guid.append(post.guid)
            pubdate.append(post.published)
            bbox.append(post.gdacs_bbox)
            coords.append(post.where)
            alevel.append(post.gdacs_episodealertlevel)
            ascore.append(post.gdacs_alertscore)
            
        guiddf = pd.DataFrame(guid)       #0
        pubdatedf = pd.DataFrame(pubdate) #1
        isodf = pd.DataFrame(iso3)        #2   
        countrydf = pd.DataFrame(country) #3
        etypedf = pd.DataFrame(etype)     #4
        epidf = pd.DataFrame(epid)        #5
        evidf = pd.DataFrame(evid)        #6
        bboxdf = pd.DataFrame(bbox)       #7
        coordf = pd.DataFrame(coords)     #8 #9
        aleveldf = pd.DataFrame(alevel)   #10
        ascoredf = pd.DataFrame(ascore)   #11
        
        frames = [guiddf, pubdatedf, isodf, countrydf, etypedf, epidf, evidf, bboxdf,coordf, aleveldf, ascoredf]
        df = pd.concat(frames, axis=1, ignore_index = True, names=[frames])
        return df[df[2].isin(wfpiso3)] #!IMPORTANT!#
        #return df
        

data = get_data()
data

#info = pd.concat([guid, title], axis=1, ignore_index=True)
#info
#info.iloc[0]

,0,1,2,3,4,5,6,7,8,9,10,11
4,VO514,"Sat, 22 Dec 2018 15:30:00 GMT",IDN,Indonesia,VO,1,514,101.423 109.423 -10.102 -2.102,"(105.423, -6.102)",Point,Red,3.58591502806943
17,DR1012698,"Mon, 24 Dec 2018 07:47:00 GMT",KHM,"Cambodia, Laos, Thailand, Vietnam",DR,4,1012698,102.699 110.699 9.9 17.9,"(106.699, 13.9)",Point,Green,0.75
21,DR1012660,"Mon, 24 Dec 2018 07:47:00 GMT",CAF,"Central African Republic, Chad",DR,6,1012660,13.623 21.623 4.123 12.123,"(17.623, 8.123)",Point,Green,0.75
22,DR1012498,"Mon, 24 Dec 2018 07:47:00 GMT",IND,"India, Pakistan",DR,9,1012498,64.889 72.889 21.391 29.391,"(68.889, 25.391)",Point,Orange,1.75
23,DR1012538,"Mon, 24 Dec 2018 07:47:00 GMT",BGD,"Bangladesh, China, India, Nepal",DR,11,1012538,76.625 84.625 16.532 24.532,"(80.625, 20.532)",Point,Orange,1.5


In [3]:
geometry = [Point(xy) for xy in zip(data[8])]
gdacsdf = data.drop([8], axis=1)
crs = {'init': 'epsg:4326'}
gdf = GeoDataFrame(gdacsdf, crs=crs, geometry=geometry)
gdf.head()

,0,1,2,3,4,5,6,7,9,10,11,geometry
4,VO514,"Sat, 22 Dec 2018 15:30:00 GMT",IDN,Indonesia,VO,1,514,101.423 109.423 -10.102 -2.102,Point,Red,3.58591502806943,POINT (105.423 -6.102)
17,DR1012698,"Mon, 24 Dec 2018 07:47:00 GMT",KHM,"Cambodia, Laos, Thailand, Vietnam",DR,4,1012698,102.699 110.699 9.9 17.9,Point,Green,0.75,POINT (106.699 13.9)
21,DR1012660,"Mon, 24 Dec 2018 07:47:00 GMT",CAF,"Central African Republic, Chad",DR,6,1012660,13.623 21.623 4.123 12.123,Point,Green,0.75,POINT (17.623 8.122999999999999)
22,DR1012498,"Mon, 24 Dec 2018 07:47:00 GMT",IND,"India, Pakistan",DR,9,1012498,64.889 72.889 21.391 29.391,Point,Orange,1.75,POINT (68.889 25.391)
23,DR1012538,"Mon, 24 Dec 2018 07:47:00 GMT",BGD,"Bangladesh, China, India, Nepal",DR,11,1012538,76.625 84.625 16.532 24.532,Point,Orange,1.5,POINT (80.625 20.532)


In [4]:
root = "http://www.gdacs.org/datareport/resources/"
eventTC = "TC/"
eventDR = "DR/"
eventEQ = "EQ/"
eventID = "1000533"
objID = "/geojson_"
objNum = "_1"
objForm = ".geojson"

tceventtest = root + eventTC + eventID + objID + eventID + objNum + objForm

print(tceventtest)

http://www.gdacs.org/datareport/resources/TC/1000533/geojson_1000533_1.geojson


Converts the GDACS data geojson xml into a geodataframe

In [5]:
#gdacsjson = gpd.read_file(tceventtest)
#gdacsjson.head(2)

Plot the geojson in folium

In [6]:
#aoi = gdacsjson.loc[gdacsjson['alertlevelepisode'] == 'Green']

m = folium.Map(tiles='stamentoner') #cartodbpositron #stamentoner #cartodbdark_matter
folium.GeoJson(gdf).add_to(m)
m.fit_bounds(m.get_bounds())
m

CHECK FOR SENTINEL DATA

In [7]:
from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
api = SentinelAPI('mgmanalili.updge', 'Suite#41', 'https://scihub.copernicus.eu/dhus')

In [8]:
#import geodaisy.converters as convert
#tc = "http://www.gdacs.org/datareport/resources/TC/1000533/geojson_1000533_3.geojson"
#wkt = convert.geojson_to_wkt(tc)

In [9]:
'''
#tc = "http://www.gdacs.org/datareport/resources/TC/1000533/geojson_1000533_3.geojson"
#dr = "http://www.gdacs.org/datareport/resources/DR/1012538/geojson_1012538_1.geojson"

#tcgpd = gpd.read_file(tc)
#drgpd = gpd.read_file(dr)

#a = tcgpd.loc[tcgpd['id'] == 'TC10005333Green']
#tcgpd

#footprint = geojson_to_wkt(read_geojson(dr))
try:
    s2 = api.query(footprint, area_relation=IsWithin,
                 date=('20181001', date(2018, 11, 25)),
                 platformname='Sentinel-2',
                 cloudcoverpercentage=(0, 50))
    
    s2_json = api.to_geojson(s2)

except:
    
    print "No image within the cloud score."

s1 = api.query(footprint, area_relation=IsWithin,
             date=('20181001', date(2018, 11, 25)),
             platformname='Sentinel-1',
               producttype='GRD',
               orbitdirection='DESCENDING') # 'ASCENDING'
s1_json = api.to_geojson(s1)

#s2_gdf = api.to_geodataframe(s2)
#s2_gdf.head(1)

#df = s2_gdf.sort_values('ingestiondate', ascending=True)
#plt.plot(s2_gdf['platformserialidentifier'],s2_gdf['ingestiondate'])
'''

'\n#tc = "http://www.gdacs.org/datareport/resources/TC/1000533/geojson_1000533_3.geojson"\n#dr = "http://www.gdacs.org/datareport/resources/DR/1012538/geojson_1012538_1.geojson"\n\n#tcgpd = gpd.read_file(tc)\n#drgpd = gpd.read_file(dr)\n\n#a = tcgpd.loc[tcgpd[\'id\'] == \'TC10005333Green\']\n#tcgpd\n\n#footprint = geojson_to_wkt(read_geojson(dr))\ntry:\n    s2 = api.query(footprint, area_relation=IsWithin,\n                 date=(\'20181001\', date(2018, 11, 25)),\n                 platformname=\'Sentinel-2\',\n                 cloudcoverpercentage=(0, 50))\n    \n    s2_json = api.to_geojson(s2)\n\nexcept:\n    \n    print "No image within the cloud score."\n\ns1 = api.query(footprint, area_relation=IsWithin,\n             date=(\'20181001\', date(2018, 11, 25)),\n             platformname=\'Sentinel-1\',\n               producttype=\'GRD\',\n               orbitdirection=\'DESCENDING\') # \'ASCENDING\'\ns1_json = api.to_geojson(s1)\n\n#s2_gdf = api.to_geodataframe(s2)\n#s2_gdf.head(1

In [8]:
#https://docs.google.com/spreadsheets/d/17HQ6FapzQCzY7YZuULudI1_kt_02Tcxxr4T5iWqPcsw/edit#gid=0
officers = '/home/michael/Desktop/gis_pips.csv'
gis = pd.read_csv(officers)
rec = gis[(gis['Regional Bureau'] == 'RBN') | (gis['Regional Bureau'] == 'RBJ') | (gis['Regional Bureau'] == 'RBB')]
rec
#len(rec)

,Regional Bureau,Country,Name,Location,Position,Email
21,RBB,Afghanistan,Wahida Azizi,Kabul,GIS Officer - Programme,wahida.azizi@wfp.org
22,RBB,Bangladesh,Mohammad Mahabubul Alam,Dhaka,GIS Officer - Programme,mohammad.mahabubul.alam@wfp.org
23,RBB,Cambodia,Chanvibol Choeur,Phnom Penh,GIS Officer - Programme,chanvibol.choeur@wfp.org
24,RBB,Fiji,Elena Diani,Suva,GIS Officer - Programme,elenoa.diani@wfp.org
25,RBB,Indonesia,Benny Istanto,Jakarta,GIS Officer - Programme,benny.istanto@wfp.org
26,RBB,Laos,Manithaphone Mahaxay,Vientiane,GIS Officer - Programme,manithaphone.mahaxay@wfp.org
27,RBB,Myanmar,Thetnaing OO,Yangoon,GIS Officer - Programme,thetnaing.oo@wfp.org
28,RBB,Nepal,Man Kshetri,Kathmandu,GIS Officer - Programme,man.kshetri@wfp.org
29,RBB,Pakistan,Iftikhar Abbas,Islamabad,GIS Officer Team Leader - Programme,iftikhar.abbas@wfp.org
30,RBB,Pakistan,Sarah Bashir,Islamabad,GIS Officer - Programme,sarah.bashir@wfp.org


In [ ]:
'''
def job():
    for url in RSS_URLS:
        feeds.append(feedparser.parse(url))

    for feed in feeds:
        for post in feed.entries:
            t = post.title
            l = post.link
            d = post.description
            w = post.where
            g = post.guid
            print t,g,d,l,w + '\n'
            s = smtplib.SMTP('smtp.gmail.com', 587)
            s.starttls()
            s.login("your.gmail.server@gmail.com", "password")
            msg = d
            sender = 'your.gmail.server@gmail.com'
            recipients = ['recipient1@gmail.com', 'recipent2@gmail.com]
            s.sendmail(sender, recipients, str(msg))
            s.quit()

#schedule.every(10).seconds.do(job)
schedule.every(24).hours.do(job)

while 1:
    schedule.run_pending()
    time.sleep(1)
'''

OTHER CODE

In [11]:
### GeoRSS
#GDACS DATA
#http://www.gdacs.org/datareport/resources/
#EONET = 'https://eonet.sci.gsfc.nasa.gov/api/v2.1/events/rss'
#GDACScap = 'www.gdacs.org/xml/gdacs_cap.xml'
#GDACS = 'http://www.gdacs.org/xml/rss.xml'
#EMS = 'http://emergency.copernicus.eu/mapping/activations-rapid/feed'
#PTC = 'https://ptwc.weather.gov/feeds/ptwc_rss_pacific.xml'

#RSS_URLS = [GDACS,EMS,PTC]

#feeds = []
#guid = []
#xy = []
#title = []
#link = []

#for url in RSS_URLS:
#    feeds.append(feedparser.parse(url))
#for feed in feeds:
#    for post in feed.entries:
#        guid.append(post.guid)
#        xy.append(post.where)
#        title.append(post.title)
#        link.append(post.link)
    #print len(guid, title, coords)
#    df_guid = pd.DataFrame(guid)
#    df_xy = pd.DataFrame(xy)
#    df_title = pd.DataFrame(title)
#    df_link = pd.DataFrame(link)
#    frames = [df_guid, df_xy, df_title, df_link]   
#    data = pd.concat([df_guid, df_xy, df_title, df_link],axis=1, ignore_index=False)

#geometry = [Point(xy) for xy in zip(data['coordinates'])]
#df = data.drop(['coordinates'], axis=1)
#crs = {'init': 'epsg:4326'}
#gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
#gdf

#geodata = gpd.GeoDataFrame(gdf)
#geodata.head(3)

#jsondata = geodata.to_json()

#m = folium.Map(tiles='stamentoner') #cartodbpositron #stamentoner #cartodbdark_matter
#folium.GeoJson(jsondata).add_to(m)

#m.fit_bounds(m.get_bounds())
#m.save(os.path.join('results', 'geopandas_2.html'))
#m

#aoi = gdacsjson.loc[gdacsjson['trackdate'] >= '01/11/2018 06:00:00']
#list(aoi)
#aoi.plot()
#----------------------------------------------------------------------------------------------------------------#
#import fiona
#fiona.drvsupport.supported_drivers['kml'] = 'rw'
#fiona.drvsupport.supported_drivers['KML'] = 'rw'
#gdacskml = gpd.read_file('http://www.gdacs.org/xml/gdacs.kml')
#gdacsjson = gpd.read_file('http://www.gdacs.org/datareport/resources/TC/1000531/geojson_1000531_27.geojson')
#print(len(gdacskml))
#gdacskml = gpd.read_file('http://www.gdacs.org/kml.aspx?') #both works
#gdacsjson.head(2)
#sample = gdacskml.loc[70]
#print(sample)